# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.117217e+02     1.554660e+00
 * time: 0.33743715286254883
     1     1.037440e+01     8.445414e-01
 * time: 2.0049779415130615
     2    -1.121620e+01     1.050138e+00
 * time: 2.6005420684814453
     3    -3.365296e+01     7.568869e-01
 * time: 3.4959909915924072
     4    -4.687883e+01     5.976787e-01
 * time: 4.335353136062622
     5    -5.666685e+01     2.447570e-01
 * time: 5.179017066955566
     6    -5.968828e+01     1.751791e-01
 * time: 5.786386013031006
     7    -6.091866e+01     7.503765e-02
 * time: 6.357948064804077
     8    -6.140836e+01     4.214580e-02
 * time: 6.929265022277832
     9    -6.170223e+01     3.050503e-02
 * time: 7.494722127914429
    10    -6.186184e+01     2.905012e-02
 * time: 8.066959142684937
    11    -6.195899e+01     2.449689e-02
 * time: 8.634283065795898
    12    -6.203424e+01     1.852507e-02
 * time: 9.224688053131104
    13    -6.209407e+01     1.388328e-02
 * time: 9.836359024047852
  

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671066
    AtomicLocal         -18.8557679
    AtomicNonlocal      14.8522650
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485369 
    Xc                  -19.3336819

    total               -62.261666461562
